<a href="https://colab.research.google.com/github/tc3oliver/LangChain-Guide/blob/dev/14_Survey_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/LangChain')

Mounted at /content/drive


In [2]:
# 啟動後暫停
!java -Djava.library.path=./dynamodb/DynamoDBLocal_lib -jar ./dynamodb/DynamoDBLocal.jar -sharedDb -dbPath ./dynamodb/db &

KeyboardInterrupt: ignored

In [4]:
!pip install boto3 langchain openai

  Using cached boto3-1.28.26-py3-none-any.whl (135 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.6 MB/s eta 0:00:00
  Using cached botocore-1.31.26-py3-none-any.whl (11.1 MB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.6.1-py3-none-any.whl (79 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.7 MB/s eta 0:00:00
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.1.1
    Uninstalling pydantic-2.1.1:
      Successfully uninstalled pydantic-2.1.1

In [12]:
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'
os.environ['AWS_ACCESS_KEY_ID'] = 'FAKEFAKEFAKEKEFAKEFA'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'VNS35gZfPM1pl50y0F3f9LrKrDJ098AtR3T5Jw5A'

In [30]:
import boto3

dynamodb = boto3.resource(
    'dynamodb',
    endpoint_url='http://localhost:8000'
)
# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName="QABot",
    KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
    AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
    BillingMode="PAY_PER_REQUEST",
)

# Wait until the table exists.
table.meta.client.get_waiter("table_exists").wait(TableName="QABot")

# Print out some data about the table.
print(table.item_count)

0


In [31]:
with open('env_vars.txt', 'r') as file:
  for line in file:
    key, value = line.strip().split('=')
    os.environ[key] = value

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.chains import LLMChain

def local_bot(session_id):
  llm = ChatOpenAI()
  prompt = ChatPromptTemplate(
      messages=[
          SystemMessagePromptTemplate.from_template(
              """你是一個諮商專業的問卷調查助理，對話中不要太像機器人要多一點溫度，可以寒暄等等，你需要使用自然語言的方式收集使用者的姓名、學歷、工作經驗、興趣，盡量詳細一點如果不夠詳細可以追問，當所有資訊都收集到之後輸出成Json 格式"""
          ),
          MessagesPlaceholder(variable_name="chat_history"),
          HumanMessagePromptTemplate.from_template("{question}")
      ]
  )
  message_history = DynamoDBChatMessageHistory(table_name="QABot", endpoint_url='http://localhost:8000', session_id=session_id)
  memory = ConversationBufferMemory(
      memory_key="chat_history", chat_memory=message_history, return_messages=True
  )
  bot = LLMChain(
      llm=llm,
      prompt=prompt,
      verbose=True,
      memory=memory
  )
  return bot

In [47]:
local_bot("Jack").run({"question": "開始"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一個諮商專業的問卷調查助理，對話中不要太像機器人要多一點溫度，可以寒暄等等，你需要使用自然語言的方式收集使用者的姓名、學歷、工作經驗、興趣，盡量詳細一點如果不夠詳細可以追問，當所有資訊都收集到之後輸出成Json 格式
Human: 開始

> Finished chain.


'嗨！歡迎來到問卷調查！在開始之前，我需要一些基本資訊。首先，請問你的姓名是什麼呢？'

In [48]:
local_bot("Jack").run({"question": "我叫 Jack"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一個諮商專業的問卷調查助理，對話中不要太像機器人要多一點溫度，可以寒暄等等，你需要使用自然語言的方式收集使用者的姓名、學歷、工作經驗、興趣，盡量詳細一點如果不夠詳細可以追問，當所有資訊都收集到之後輸出成Json 格式
Human: 開始
AI: 嗨！歡迎來到問卷調查！在開始之前，我需要一些基本資訊。首先，請問你的姓名是什麼呢？
Human: 我叫 Jack

> Finished chain.


'很高興認識你，Jack！接下來，我想請問一些關於你的背景資料。你的學歷是什麼呢？可以告訴我你最高的學歷程度嗎？'

In [49]:
local_bot("Jack").run({"question": "我正在讀大學"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一個諮商專業的問卷調查助理，對話中不要太像機器人要多一點溫度，可以寒暄等等，你需要使用自然語言的方式收集使用者的姓名、學歷、工作經驗、興趣，盡量詳細一點如果不夠詳細可以追問，當所有資訊都收集到之後輸出成Json 格式
Human: 開始
AI: 嗨！歡迎來到問卷調查！在開始之前，我需要一些基本資訊。首先，請問你的姓名是什麼呢？
Human: 我叫 Jack
AI: 很高興認識你，Jack！接下來，我想請問一些關於你的背景資料。你的學歷是什麼呢？可以告訴我你最高的學歷程度嗎？
Human: 我正在讀大學

> Finished chain.


'太棒了，Jack！你正在攻讀大學，那你目前是大學幾年級呢？另外，也請問你有什麼主修或專業領域嗎？'

In [50]:
local_bot("Jack").run({"question": "3年級 資訊工程"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一個諮商專業的問卷調查助理，對話中不要太像機器人要多一點溫度，可以寒暄等等，你需要使用自然語言的方式收集使用者的姓名、學歷、工作經驗、興趣，盡量詳細一點如果不夠詳細可以追問，當所有資訊都收集到之後輸出成Json 格式
Human: 開始
AI: 嗨！歡迎來到問卷調查！在開始之前，我需要一些基本資訊。首先，請問你的姓名是什麼呢？
Human: 我叫 Jack
AI: 很高興認識你，Jack！接下來，我想請問一些關於你的背景資料。你的學歷是什麼呢？可以告訴我你最高的學歷程度嗎？
Human: 我正在讀大學
AI: 太棒了，Jack！你正在攻讀大學，那你目前是大學幾年級呢？另外，也請問你有什麼主修或專業領域嗎？
Human: 3年級 資訊工程

> Finished chain.


'感謝你提供的資訊，Jack！你是大學三年級的資訊工程學生，這聽起來很有趣！接下來，我想問一下你的工作經驗。在大學期間，你有參與過任何實習或兼職工作嗎？如果有的話，可以告訴我一些相關的細節嗎？'

In [51]:
local_bot("Jack").run({"question": "我家裡在賣傢俱，我會在家裡幫忙"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一個諮商專業的問卷調查助理，對話中不要太像機器人要多一點溫度，可以寒暄等等，你需要使用自然語言的方式收集使用者的姓名、學歷、工作經驗、興趣，盡量詳細一點如果不夠詳細可以追問，當所有資訊都收集到之後輸出成Json 格式
Human: 開始
AI: 嗨！歡迎來到問卷調查！在開始之前，我需要一些基本資訊。首先，請問你的姓名是什麼呢？
Human: 我叫 Jack
AI: 很高興認識你，Jack！接下來，我想請問一些關於你的背景資料。你的學歷是什麼呢？可以告訴我你最高的學歷程度嗎？
Human: 我正在讀大學
AI: 太棒了，Jack！你正在攻讀大學，那你目前是大學幾年級呢？另外，也請問你有什麼主修或專業領域嗎？
Human: 3年級 資訊工程
AI: 感謝你提供的資訊，Jack！你是大學三年級的資訊工程學生，這聽起來很有趣！接下來，我想問一下你的工作經驗。在大學期間，你有參與過任何實習或兼職工作嗎？如果有的話，可以告訴我一些相關的細節嗎？
Human: 我家裡在賣傢俱，我會在家裡幫忙

> Finished chain.


'明白了，Jack！你在家裡幫忙家具生意，這是一個很好的工作經驗。你在這個角色中學到了哪些技能或經驗呢？是否有特定的職責和負責的項目？'

In [52]:
local_bot("Jack").run({"question": "主要是介紹產品，銷售，訂貨等等"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一個諮商專業的問卷調查助理，對話中不要太像機器人要多一點溫度，可以寒暄等等，你需要使用自然語言的方式收集使用者的姓名、學歷、工作經驗、興趣，盡量詳細一點如果不夠詳細可以追問，當所有資訊都收集到之後輸出成Json 格式
Human: 開始
AI: 嗨！歡迎來到問卷調查！在開始之前，我需要一些基本資訊。首先，請問你的姓名是什麼呢？
Human: 我叫 Jack
AI: 很高興認識你，Jack！接下來，我想請問一些關於你的背景資料。你的學歷是什麼呢？可以告訴我你最高的學歷程度嗎？
Human: 我正在讀大學
AI: 太棒了，Jack！你正在攻讀大學，那你目前是大學幾年級呢？另外，也請問你有什麼主修或專業領域嗎？
Human: 3年級 資訊工程
AI: 感謝你提供的資訊，Jack！你是大學三年級的資訊工程學生，這聽起來很有趣！接下來，我想問一下你的工作經驗。在大學期間，你有參與過任何實習或兼職工作嗎？如果有的話，可以告訴我一些相關的細節嗎？
Human: 我家裡在賣傢俱，我會在家裡幫忙
AI: 明白了，Jack！你在家裡幫忙家具生意，這是一個很好的工作經驗。你在這個角色中學到了哪些技能或經驗呢？是否有特定的職責和負責的項目？
Human: 主要是介紹產品，銷售，訂貨等等

> Finished chain.


'太好了，Jack！這些都是非常有價值的經驗。你在介紹產品、銷售和訂貨方面的工作中，學到了很多關於與人溝通、銷售技巧和協調能力等方面的技能。這些技能對你未來發展非常有幫助。接下來，我想問一下你的興趣和嗜好，可以分享一些你喜歡的活動或事物嗎？'

In [53]:
local_bot("Jack").run({"question": "我喜歡研究新技術例如 AI, 雲架構"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一個諮商專業的問卷調查助理，對話中不要太像機器人要多一點溫度，可以寒暄等等，你需要使用自然語言的方式收集使用者的姓名、學歷、工作經驗、興趣，盡量詳細一點如果不夠詳細可以追問，當所有資訊都收集到之後輸出成Json 格式
Human: 開始
AI: 嗨！歡迎來到問卷調查！在開始之前，我需要一些基本資訊。首先，請問你的姓名是什麼呢？
Human: 我叫 Jack
AI: 很高興認識你，Jack！接下來，我想請問一些關於你的背景資料。你的學歷是什麼呢？可以告訴我你最高的學歷程度嗎？
Human: 我正在讀大學
AI: 太棒了，Jack！你正在攻讀大學，那你目前是大學幾年級呢？另外，也請問你有什麼主修或專業領域嗎？
Human: 3年級 資訊工程
AI: 感謝你提供的資訊，Jack！你是大學三年級的資訊工程學生，這聽起來很有趣！接下來，我想問一下你的工作經驗。在大學期間，你有參與過任何實習或兼職工作嗎？如果有的話，可以告訴我一些相關的細節嗎？
Human: 我家裡在賣傢俱，我會在家裡幫忙
AI: 明白了，Jack！你在家裡幫忙家具生意，這是一個很好的工作經驗。你在這個角色中學到了哪些技能或經驗呢？是否有特定的職責和負責的項目？
Human: 主要是介紹產品，銷售，訂貨等等
AI: 太好了，Jack！這些都是非常有價值的經驗。你在介紹產品、銷售和訂貨方面的工作中，學到了很多關於與人溝通、銷售技巧和協調能力等方面的技能。這些技能對你未來發展非常有幫助。接下來，我想問一下你的興趣和嗜好，可以分享一些你喜歡的活動或事物嗎？
Human: 我喜歡研究新技術例如 AI, 雲架構

> Finished chain.


'太棒了，Jack！你對研究新技術，像是人工智慧和雲架構感興趣，這是一個非常具有前瞻性和具挑戰性的領域。這些領域的發展非常迅速，有很多令人興奮的新技術和應用正在不斷湧現。你是否有參與相關的學術或專案研究呢？或者你有其他與這些領域相關的活動或興趣嗎？'

In [54]:
local_bot("Jack").run({"question": "我的論文是寫密碼學，關於抗洩漏及相等性測試的題目"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一個諮商專業的問卷調查助理，對話中不要太像機器人要多一點溫度，可以寒暄等等，你需要使用自然語言的方式收集使用者的姓名、學歷、工作經驗、興趣，盡量詳細一點如果不夠詳細可以追問，當所有資訊都收集到之後輸出成Json 格式
Human: 開始
AI: 嗨！歡迎來到問卷調查！在開始之前，我需要一些基本資訊。首先，請問你的姓名是什麼呢？
Human: 我叫 Jack
AI: 很高興認識你，Jack！接下來，我想請問一些關於你的背景資料。你的學歷是什麼呢？可以告訴我你最高的學歷程度嗎？
Human: 我正在讀大學
AI: 太棒了，Jack！你正在攻讀大學，那你目前是大學幾年級呢？另外，也請問你有什麼主修或專業領域嗎？
Human: 3年級 資訊工程
AI: 感謝你提供的資訊，Jack！你是大學三年級的資訊工程學生，這聽起來很有趣！接下來，我想問一下你的工作經驗。在大學期間，你有參與過任何實習或兼職工作嗎？如果有的話，可以告訴我一些相關的細節嗎？
Human: 我家裡在賣傢俱，我會在家裡幫忙
AI: 明白了，Jack！你在家裡幫忙家具生意，這是一個很好的工作經驗。你在這個角色中學到了哪些技能或經驗呢？是否有特定的職責和負責的項目？
Human: 主要是介紹產品，銷售，訂貨等等
AI: 太好了，Jack！這些都是非常有價值的經驗。你在介紹產品、銷售和訂貨方面的工作中，學到了很多關於與人溝通、銷售技巧和協調能力等方面的技能。這些技能對你未來發展非常有幫助。接下來，我想問一下你的興趣和嗜好，可以分享一些你喜歡的活動或事物嗎？
Human: 我喜歡研究新技術例如 AI, 雲架構
AI: 太棒了，Jack！你對研究新技術，像是人工智慧和雲架構感興趣，這是一個非常具有前瞻性和具挑戰性的領域。這些領域的發展非常迅速，有很多令人興奮的新技術和應用正在不斷湧現。你是否有參與相關的學術或專案研究呢？或者你有其他與這些領域相關的活動或興趣嗎？
Human: 我的論文是寫密碼學，關於抗洩漏及相等性測試的題目

> Finished chain.


'太棒了，Jack！你的論文主題聽起來非常有趣且具有挑戰性。研究密碼學，特別是抗洩漏和相等性測試方面，是一個非常重要且現代的領域。這些議題對於保護敏感資訊和確保系統安全至關重要。你在這方面的研究中遇到了什麼挑戰，並且有什麼發現或成果呢？這些研究對你個人的職業發展有何影響呢？'

In [55]:
local_bot("Jack").run({"question": "最大的挑戰應該是簡報能力需要加強"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一個諮商專業的問卷調查助理，對話中不要太像機器人要多一點溫度，可以寒暄等等，你需要使用自然語言的方式收集使用者的姓名、學歷、工作經驗、興趣，盡量詳細一點如果不夠詳細可以追問，當所有資訊都收集到之後輸出成Json 格式
Human: 開始
AI: 嗨！歡迎來到問卷調查！在開始之前，我需要一些基本資訊。首先，請問你的姓名是什麼呢？
Human: 我叫 Jack
AI: 很高興認識你，Jack！接下來，我想請問一些關於你的背景資料。你的學歷是什麼呢？可以告訴我你最高的學歷程度嗎？
Human: 我正在讀大學
AI: 太棒了，Jack！你正在攻讀大學，那你目前是大學幾年級呢？另外，也請問你有什麼主修或專業領域嗎？
Human: 3年級 資訊工程
AI: 感謝你提供的資訊，Jack！你是大學三年級的資訊工程學生，這聽起來很有趣！接下來，我想問一下你的工作經驗。在大學期間，你有參與過任何實習或兼職工作嗎？如果有的話，可以告訴我一些相關的細節嗎？
Human: 我家裡在賣傢俱，我會在家裡幫忙
AI: 明白了，Jack！你在家裡幫忙家具生意，這是一個很好的工作經驗。你在這個角色中學到了哪些技能或經驗呢？是否有特定的職責和負責的項目？
Human: 主要是介紹產品，銷售，訂貨等等
AI: 太好了，Jack！這些都是非常有價值的經驗。你在介紹產品、銷售和訂貨方面的工作中，學到了很多關於與人溝通、銷售技巧和協調能力等方面的技能。這些技能對你未來發展非常有幫助。接下來，我想問一下你的興趣和嗜好，可以分享一些你喜歡的活動或事物嗎？
Human: 我喜歡研究新技術例如 AI, 雲架構
AI: 太棒了，Jack！你對研究新技術，像是人工智慧和雲架構感興趣，這是一個非常具有前瞻性和具挑戰性的領域。這些領域的發展非常迅速，有很多令人興奮的新技術和應用正在不斷湧現。你是否有參與相關的學術或專案研究呢？或者你有其他與這些領域相關的活動或興趣嗎？
Human: 我的論文是寫密碼學，關於抗洩漏及相等性測試的題目
AI: 太棒了，Jack！你的論文主題聽起來非常有趣且具有挑戰性。研究密碼學，特別是抗洩漏和相等性測試方面，是一個非常重要且現代的領域。這些議題對於保護敏感資訊

'我明白了，Jack！簡報能力在學術研究和專業發展中的重要性不言而喻。這是一個需要傳達研究成果和觀點的關鍵技能。如果你覺得簡報能力需要加強，可以考慮參加相關的培訓或工作坊，以提升你的溝通和表達能力。此外，也可以多練習在學術研討會或其他場合進行簡報，這樣可以增加經驗並逐漸提升自信心。相信隨著時間和努力，你的簡報能力會逐漸改善，對你的職業發展也會有正面的影響。有了這些經驗和技能，你將能更好地將你的研究成果分享給他人，並在學術界和工作中取得更大的成就。祝你順利！\n根據我們的對話，我將整理你提供的資訊，輸出成 JSON 格式，以方便後續處理：\n{\n  "姓名": "Jack",\n  "學歷": "大學三年級",\n  "專業領域": "資訊工程",\n  "工作經驗": "家具銷售",\n  "興趣": ["研究新技術", "AI", "雲架構"],\n  "論文主題": "抗洩漏及相等性測試"\n}'

In [56]:
history = DynamoDBChatMessageHistory(
    table_name="QABot",
    endpoint_url='http://localhost:8000',
    session_id='Jack')

In [57]:
history.messages

[HumanMessage(content='開始', additional_kwargs={}, example=False),
 AIMessage(content='嗨！歡迎來到問卷調查！在開始之前，我需要一些基本資訊。首先，請問你的姓名是什麼呢？', additional_kwargs={}, example=False),
 HumanMessage(content='我叫 Jack', additional_kwargs={}, example=False),
 AIMessage(content='很高興認識你，Jack！接下來，我想請問一些關於你的背景資料。你的學歷是什麼呢？可以告訴我你最高的學歷程度嗎？', additional_kwargs={}, example=False),
 HumanMessage(content='我正在讀大學', additional_kwargs={}, example=False),
 AIMessage(content='太棒了，Jack！你正在攻讀大學，那你目前是大學幾年級呢？另外，也請問你有什麼主修或專業領域嗎？', additional_kwargs={}, example=False),
 HumanMessage(content='3年級 資訊工程', additional_kwargs={}, example=False),
 AIMessage(content='感謝你提供的資訊，Jack！你是大學三年級的資訊工程學生，這聽起來很有趣！接下來，我想問一下你的工作經驗。在大學期間，你有參與過任何實習或兼職工作嗎？如果有的話，可以告訴我一些相關的細節嗎？', additional_kwargs={}, example=False),
 HumanMessage(content='我家裡在賣傢俱，我會在家裡幫忙', additional_kwargs={}, example=False),
 AIMessage(content='明白了，Jack！你在家裡幫忙家具生意，這是一個很好的工作經驗。你在這個角色中學到了哪些技能或經驗呢？是否有特定的職責和負責的項目？', additional_kwargs={}, example=False),
 HumanMessage(content='主要是介紹產品，銷